In [47]:
import pandas as pd

df = pd.read_csv('data/2023_2024.csv')
df.head()
# remove NaN username
df = df.dropna(subset=['username'])
df.head()

,datetime,username,message
1,2022년 9월 26일 오후 2:45,애교뿜뿜 어피치,넵넵 논의 가능 합니다! 연락 한 번 부탁드립니다 ㅎㅎ\n\n
2,2022년 9월 27일 오전 10:19,떨고있는 어피치,한국어 자연어 처리 관련 좋은 깃헙 레포 있을까요 구글링하면 어느정도 나오긴하는데...
3,2022년 9월 27일 오전 10:19,떨고있는 어피치,미디엄이나 블로그 등도 좋아요!\n
4,2022년 9월 27일 오후 1:07,볼츠만6대손,오 named tensor 를 torch 에서도 지원해주는군요\nhttps://p...
5,2022년 9월 27일 오후 1:07,볼츠만6대손,사진\n


In [48]:
# convert datetime 
# 2022년 9월 27일 오후 1:07 -> 2022-09-27 13:07:00
# substitute '오전' to 'AM', '오후' to 'PM'
df['datetime'] = df['datetime'].str.replace('오전', 'AM')
df['datetime'] = df['datetime'].str.replace('오후', 'PM')
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y년 %m월 %d일 %p %I:%M')

In [49]:
df

,datetime,username,message
1,2022-09-26 14:45:00,애교뿜뿜 어피치,넵넵 논의 가능 합니다! 연락 한 번 부탁드립니다 ㅎㅎ\n\n
2,2022-09-27 10:19:00,떨고있는 어피치,한국어 자연어 처리 관련 좋은 깃헙 레포 있을까요 구글링하면 어느정도 나오긴하는데...
3,2022-09-27 10:19:00,떨고있는 어피치,미디엄이나 블로그 등도 좋아요!\n
4,2022-09-27 13:07:00,볼츠만6대손,오 named tensor 를 torch 에서도 지원해주는군요\nhttps://p...
5,2022-09-27 13:07:00,볼츠만6대손,사진\n
...,...,...,...
33616,2024-01-04 00:41:00,앤드류엉/10년차/백수희망자,ㅇㅇ 넵\n
33617,2024-01-04 00:41:00,앤드류엉/10년차/백수희망자,왜 비싸게 학습해서 탄소낭비하면서 리더보드 하냐\n
33618,2024-01-04 00:41:00,머린이,아닠ㅋㅋㅋㅋㅋ 작명 미쳤네요\n
33619,2024-01-04 00:41:00,앤드류엉/10년차/백수희망자,3등도 저희 모델이에요\n


In [50]:
# subset datetime by 2023
df = df[df['datetime'] < '2024-01-01']
df = df[df['datetime'] >= '2023-01-01']

In [51]:
# top 10 users with most chats
df['username'].value_counts().head(10)
# plot with plotly
import plotly.express as px
fig = px.bar(df['username'].value_counts().head(10), title='2023 소통냥이 TOP 10!')
# remove legend
fig.update_layout(showlegend=False)
# y axis name to '채팅 수'
fig.update_yaxes(title_text='채팅 수')

# plot it as time progresses.
import plotly.graph_objects as go
import numpy as np

top10 = df['username'].value_counts().head(10).index
df['date'] = df['datetime'].dt.date
df_forplot = df.groupby(['username', 'date']).size().reset_index(name='counts')
df_forplot.head()
# plot number of cumulative chats per day
fig = go.Figure()
for username in top10:
    fig.add_trace(go.Scatter(x=df_forplot[df_forplot['username'] == username]['date'],
                             y=np.cumsum(df_forplot[df_forplot['username'] == username]['counts']),
                             mode='lines',
                             name=username))
fig.update_layout(title='2023 소통냥이 TOP 10!',
                    xaxis_title='날짜',
                    yaxis_title='채팅 수')
fig.show()

In [52]:
# make have_link column
df['have_link'] = df['message'].str.contains('http')

# make chat_length column
df['chat_length'] = df['message'].str.len()

# make long_chat_with_link column : long chat with link
df['long_chat_with_link'] = (df['chat_length'] >= 100) & df['have_link']

df[df['long_chat_with_link'] == True]['username'].value_counts().head(10)
# # plot with plotly
# import plotly.express as px
# fig = px.bar(df[df['long_chat_with_link'] == True]['username'].value_counts().head(10), title='2023 정보냥이 TOP 10!')
# fig.update_layout(showlegend=False)
# fig.update_yaxes(title_text='설명과 함께 공유한 링크 수')
# fig.show()
top10 = df[df['long_chat_with_link'] == True]['username'].value_counts().head(10).index
df_forplot = df[df['long_chat_with_link'] == True].groupby(['username', 'date']).size().reset_index(name='counts')
df_forplot.head()
# plot number of cumulative chats per day
fig = go.Figure()
for username in top10:
    fig.add_trace(go.Scatter(x=df_forplot[df_forplot['username'] == username]['date'],
                             y=np.cumsum(df_forplot[df_forplot['username'] == username]['counts']),
                             mode='lines',
                             name=username))
fig.update_layout(title='2023 정보냥이 TOP 10!',
                    xaxis_title='날짜',
                    yaxis_title='설명과 함께 공유한 링크 수')
fig.show()

In [53]:
# make have_arxiv column
df['have_arxiv'] = df['message'].str.contains('https://arxiv.org/')


# plot with plotly
import plotly.express as px
fig = px.bar(df[df['have_arxiv'] == True]['username'].value_counts().head(10), title='2023 Arxiv 냥이 TOP 10!')
fig.update_layout(showlegend=False)
fig.update_yaxes(title_text='공유한 Arxiv 링크 수')
fig.show()

top10 = df[df['have_arxiv'] == True]['username'].value_counts().head(10).index
df_forplot = df[df['have_arxiv'] == True].groupby(['username', 'date']).size().reset_index(name='counts')
df_forplot.head()
# plot number of cumulative chats per day
fig = go.Figure()
for username in top10:
    fig.add_trace(go.Scatter(x=df_forplot[df_forplot['username'] == username]['date'],
                             y=np.cumsum(df_forplot[df_forplot['username'] == username]['counts']),
                             mode='lines',
                             name=username))
fig.update_layout(title='2023 Arxiv 냥이 TOP 10!',
                    xaxis_title='날짜',
                    yaxis_title='공유한 Arxiv 링크 수')
fig.show()


In [44]:
# list of arxiv links shared
arxiv_lists = df[df['have_arxiv'] == True]['message'].str.extract(r'(https://arxiv.org/[^ ]*)')[0].unique()
# further strip the link
arxiv_lists = [x.split('\n')[0].strip() for x in arxiv_lists]
# all of the links start with http and ends with .pdf or XXXX (4 digits). capture those with re

import re
arxiv_lists = [x for x in arxiv_lists if re.match(r'http.*\d{4}$', x) or re.match(r'http.*pdf$', x)]

links = []
for x in arxiv_lists:
    # have stuff that ends with pdf
    if re.match(r'http.*pdf$', x):
        new_link = re.sub(r'http.*pdf$', r'\g<0>', x)
        new_link = new_link.replace('.pdf', '').replace('pdf', 'abs')
        
        links.append(new_link)
    # have stuff that ends with 4 digits
    elif re.match(r'http.*\d{4}$', x):
        new_link = re.sub(r'http.*\d{4}$', r'\g<0>', x)
        links.append(new_link)
    else:
        assert False
        
df = pd.DataFrame(links, columns=['link'])
df.to_csv('data/arxiv_links_2023.csv', index=False)